In [ ]:
!pip install -U transformers torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 38.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 67.4 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

In [ ]:
model_id= 'naver/splade-cocondenser-ensembledistil'

In [ ]:
tokenizer=AutoTokenizer.from_pretrained(model_id)
model=AutoModelForMaskedLM.from_pretrained(model_id)

In [ ]:
text="There are no humpback whales found in the thick and dense rainforests of South Africa"

In [ ]:
tokens=tokenizer(text, return_tensors='pt')
output=model(**tokens)
output

MaskedLMOutput(loss=None, logits=tensor([[[ -5.7134,  -8.4089,  -7.4289,  ...,  -7.7797,  -7.4996,  -4.7942],
         [ -6.2479,  -8.8598,  -7.8209,  ...,  -8.2088,  -7.8196,  -5.1440],
         [ -6.2644,  -8.8962,  -7.7724,  ...,  -8.2235,  -7.7365,  -5.2182],
         ...,
         [ -8.7042,  -9.0063,  -9.1119,  ...,  -8.8412,  -8.5455,  -5.9537],
         [ -9.1273,  -9.8217,  -9.3334,  ...,  -9.3996,  -9.4323,  -6.6747],
         [-20.0070, -16.5636, -16.0569,  ..., -15.7360, -15.4313, -16.8862]]],
       grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [ ]:
tokens.attention_mask

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])

We can see the subsequent tokens used for the model further.

In [ ]:
tokenizer.convert_ids_to_tokens(tokens["input_ids"][0])

['[CLS]',
 'there',
 'are',
 'no',
 'hum',
 '##p',
 '##back',
 'whales',
 'found',
 'in',
 'the',
 'thick',
 'and',
 'dense',
 'rainforest',
 '##s',
 'of',
 'south',
 'africa',
 '[SEP]']

In [ ]:
output.logits.shape

torch.Size([1, 19, 30522])

I have just used a random sentence for text, and correspondingly, it leaves us with 19 probability distributions, each with dimensionality 30522.

In [ ]:
import torch

vec=torch.max(
    torch.log(
        1+torch.relu(output.logits)
    ) * tokens.attention_mask.unsqueeze(-1), dim=1
)[0].squeeze()

In [ ]:
vec.shape

torch.Size([30522])

In [ ]:
vec

tensor([0., 0., 0.,  ..., 0., 0., 0.], grad_fn=<SqueezeBackward0>)

In [ ]:
cols=vec.nonzero().squeeze().cpu().tolist()
len(cols)

67

In [ ]:
weights=vec[cols].cpu().tolist()
sparse_dict=dict(zip(cols,weights))

In [ ]:
sparse_dict

{1999: 0.08050337433815002,
 2024: 0.22472348809242249,
 2025: 0.3108704686164856,
 2027: 0.4775053858757019,
 2030: 1.0757325887680054,
 2045: 1.1077091693878174,
 2053: 1.3516221046447754,
 2069: 0.20165568590164185,
 2078: 0.3278961777687073,
 2148: 1.2895724773406982,
 2179: 0.028736703097820282,
 2182: 0.03424646705389023,
 2314: 0.018253089860081673,
 2380: 0.06938674300909042,
 2427: 0.7120238542556763,
 2444: 0.3209882378578186,
 2498: 0.1689511090517044,
 2512: 0.24999350309371948,
 2748: 0.7983103394508362,
 3060: 1.2565898895263672,
 3088: 1.5747015476226807,
 3137: 0.21118004620075226,
 3224: 1.1911038160324097,
 3295: 0.031008141115307808,
 3739: 0.28845152258872986,
 3768: 0.3027591109275818,
 3899: 0.0035886666737496853,
 3904: 0.8028383851051331,
 4019: 0.1178249940276146,
 4044: 0.15320564806461334,
 4111: 0.24388985335826874,
 4176: 0.18476220965385437,
 4224: 0.0505230538547039,
 4317: 1.3528296947479248,
 4743: 0.04358520358800888,
 4839: 0.592359185218811,
 5081: 0

In [ ]:
idx2token = {
    idx: token for token, idx in tokenizer.get_vocab().items()
}

In [ ]:
sparse_dict_tokens = {
    idx2token[idx]: round(weight, 2) for idx, weight in zip(cols, weights)
}
sparse_dict_tokens = {
    k: v for k, v in sorted(
        sparse_dict_tokens.items(),
        key=lambda item: item[1],
        reverse=True
    )
}
sparse_dict_tokens

{'africa': 1.57,
 '##uta': 1.52,
 '##ang': 1.5,
 'rainforest': 1.38,
 'dense': 1.36,
 'no': 1.35,
 'thick': 1.35,
 'south': 1.29,
 'african': 1.26,
 'forest': 1.19,
 'habitat': 1.14,
 'there': 1.11,
 'or': 1.08,
 'yes': 0.8,
 'none': 0.8,
 'species': 0.71,
 'sa': 0.64,
 'exist': 0.59,
 'desert': 0.59,
 'elephant': 0.54,
 'they': 0.48,
 'extinction': 0.37,
 'zoo': 0.34,
 '##n': 0.33,
 'live': 0.32,
 'not': 0.31,
 'lack': 0.3,
 'presence': 0.29,
 'cave': 0.29,
 'threat': 0.28,
 'non': 0.25,
 'animal': 0.24,
 'breed': 0.24,
 'are': 0.22,
 'hunting': 0.22,
 'mountain': 0.21,
 'moose': 0.21,
 'only': 0.2,
 'animals': 0.18,
 'monkey': 0.18,
 'nothing': 0.17,
 'tiger': 0.17,
 'geography': 0.16,
 'environment': 0.15,
 'volcano': 0.14,
 'fauna': 0.13,
 'escape': 0.12,
 'shelter': 0.1,
 'in': 0.08,
 'park': 0.07,
 'terrain': 0.07,
 'conservation': 0.06,
 'tribe': 0.06,
 'zone': 0.05,
 'behavior': 0.05,
 'hide': 0.05,
 'belong': 0.05,
 'snake': 0.05,
 'bird': 0.04,
 'guinea': 0.04,
 'endangered':

We can now use it to compare vectors

In [ ]:
texts = [
   "Programmed cell death (PCD) is the regulated death of cells within an organism",
   "How is the scheduled death of cells within a living thing regulated?",
   "Photosynthesis is the process of storing light energy as chemical energy in cells"
]

In [ ]:
tokens = tokenizer(
    texts, return_tensors='pt',
    padding=True, truncation=True
)
output = model(**tokens)
vecs = torch.max(
    torch.log(1 + torch.relu(output.logits)) * tokens.attention_mask.unsqueeze(-1), dim=1
)[0].squeeze().detach().cpu().numpy()
vecs.shape

(3, 30522)